In [1]:
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import datetime
import random
import re
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
import requests
import re
import py2neo as neo
import pandas as pd
import numpy as np
import random

In [2]:
Group=[[] for _ in range(10)]
teams=[[] for _ in range(10)]
country=[[] for _ in range(10)]

In [3]:
def creat_seasons_link(i):
    year=2021-i
    if year<=2007:
        year=year-1
    return ("https://www.uefa.com/uefachampionsleague/history/seasons/"+str(year)+"/")
    

In [4]:
def get_nation(link):
    link="https://www.uefa.com"+link
    html = urlopen(link)
    bs = BeautifulSoup(html, 'html.parser')
    return bs.find("span",{'class': 'team-country-name'}).get_text()

In [5]:
def get_seasons(link,i):
    print(link)
    print(i)
    html = urlopen(link)
    bs = BeautifulSoup(html, 'html.parser')
    divs=bs.findAll("div",{'class': 'carousel-item'})
    for data in divs:
        Group[i].append(data.find("p").get_text().split("L")[0])
        for a in data.findAll("a",{'class':'table_team-name_block'}):
            country[i].append(get_nation(a.get("href")) )
        for span in data.findAll("span",{'class':'team-name js-team-name'}):
            teams[i].append(span.get_text().split("Playing now")[0])
    

In [6]:
for i in range(0,8):
    get_seasons(creat_seasons_link(i),i)

https://www.uefa.com/uefachampionsleague/history/seasons/2021/
0
https://www.uefa.com/uefachampionsleague/history/seasons/2020/
1
https://www.uefa.com/uefachampionsleague/history/seasons/2019/
2
https://www.uefa.com/uefachampionsleague/history/seasons/2018/
3
https://www.uefa.com/uefachampionsleague/history/seasons/2017/
4
https://www.uefa.com/uefachampionsleague/history/seasons/2016/
5
https://www.uefa.com/uefachampionsleague/history/seasons/2015/
6
https://www.uefa.com/uefachampionsleague/history/seasons/2014/
7


In [10]:
def retest(text):
    return re.sub(r'[^A-Za-z0-9]'," ",text).strip()

In [11]:
for n in range(0,8):
    for i in range(0,len(Group[n])):
        Group[n][i]=retest(Group[n][i])

    for i in range(0,len(teams[n])):
        teams[n][i]=retest(teams[n][i])

    for i in range(0,len(country[n])):
        country[n][i]=retest(country[n][i]) 


In [15]:
country[1]

['FRA',
 'ESP',
 'BEL',
 'TUR',
 'GER',
 'ENG',
 'GRE',
 'SRB',
 'ENG',
 'ITA',
 'UKR',
 'CRO',
 'ITA',
 'ESP',
 'GER',
 'RUS',
 'ENG',
 'ITA',
 'AUT',
 'BEL',
 'ESP',
 'GER',
 'ITA',
 'CZE',
 'GER',
 'FRA',
 'POR',
 'RUS',
 'ESP',
 'ENG',
 'NED',
 'FRA']

In [16]:
spans_team=[[] for _ in range(10)]
spans_HS=[[] for _ in range(10)]
spans_AS=[[] for _ in range(10)]
match_count=[[] for _ in range(10)]

In [17]:
for n in range(0,8):
    match_count[n].append( int (len(teams[n])/4*12) ) 
    team_count= int( len(teams[n])/4*2 )
    while team_count!=2:
        match_count[n][0]=match_count[n][0]+team_count
        team_count=team_count/2
    match_count[n][0]= int (match_count[n][0]+1)


In [18]:
def creat_match_link(year,indx):
    return ("https://www.uefa.com/uefachampionsleague/history/seasons/"
           +year
           +"/matches/libraries/matches-by-team/index_"
           +str(indx))

In [20]:
def get_Data(link,n):
    print(link)
    html = urlopen(link)
    bs = BeautifulSoup(html, 'html.parser')
    spans_team[n].extend(bs.findAll("span",{'class': 'js-fitty'}))
    spans_HS[n].extend(bs.findAll("span",{'class': 'js-team--home-score home-score'}))
    spans_AS[n].extend(bs.findAll("span",{'class': 'js-team--away-score away-score'}))  

In [21]:
for n in range(0,8):
    year=2021-n
    if year<=2007:
        year=year-1
    for i in range(0,3):
        get_Data(creat_match_link(str(year),i),n)    


https://www.uefa.com/uefachampionsleague/history/seasons/2021/matches/libraries/matches-by-team/index_0
https://www.uefa.com/uefachampionsleague/history/seasons/2021/matches/libraries/matches-by-team/index_1
https://www.uefa.com/uefachampionsleague/history/seasons/2021/matches/libraries/matches-by-team/index_2
https://www.uefa.com/uefachampionsleague/history/seasons/2020/matches/libraries/matches-by-team/index_0
https://www.uefa.com/uefachampionsleague/history/seasons/2020/matches/libraries/matches-by-team/index_1
https://www.uefa.com/uefachampionsleague/history/seasons/2020/matches/libraries/matches-by-team/index_2
https://www.uefa.com/uefachampionsleague/history/seasons/2019/matches/libraries/matches-by-team/index_0
https://www.uefa.com/uefachampionsleague/history/seasons/2019/matches/libraries/matches-by-team/index_1
https://www.uefa.com/uefachampionsleague/history/seasons/2019/matches/libraries/matches-by-team/index_2
https://www.uefa.com/uefachampionsleague/history/seasons/2018/ma

In [25]:
match=[[] for _ in range(10)]
n=0

In [27]:
for x in range(0,8):
    n=0
    for i in range(0, int(match_count[x][0]) ):
        if int(spans_HS[x][i].get_text())>int(spans_AS[x][i].get_text()):
            match[x].append(retest(spans_team[x][n].get_text()) 
                 +"#H#"
                 +retest(spans_HS[x][i].get_text())
                 +"-"
                 +retest(spans_AS[x][i].get_text())
                 +"#A#"
                 +retest(spans_team[x][n+1].get_text())
                 +" Win "+retest(spans_team[x][n].get_text()) )
        else:
            match[x].append(retest(spans_team[x][n].get_text()) 
                 +"#H#"
                 +retest(spans_HS[x][i].get_text())
                 +"-"
                 +retest(spans_AS[x][i].get_text())
                 +"#A#"
                 +retest(spans_team[x][n+1].get_text())
                 +" Win "+retest(spans_team[x][n+1].get_text()) )
        n=n+2 

In [30]:
match[0]

['Man  City#H#0-1#A#Chelsea Win Chelsea',
 'Chelsea#H#2-0#A#Real Madrid Win Chelsea',
 'Man  City#H#2-0#A#Paris Win Man  City',
 'Paris#H#1-2#A#Man  City Win Man  City',
 'Real Madrid#H#1-1#A#Chelsea Win Chelsea',
 'Dortmund#H#1-2#A#Man  City Win Man  City',
 'Liverpool#H#0-0#A#Real Madrid Win Real Madrid',
 'Chelsea#H#0-1#A#Porto Win Porto',
 'Paris#H#0-1#A#Bayern Win Bayern',
 'Bayern#H#2-3#A#Paris Win Paris',
 'Porto#H#0-2#A#Chelsea Win Chelsea',
 'Real Madrid#H#3-1#A#Liverpool Win Real Madrid',
 'Man  City#H#2-1#A#Dortmund Win Man  City',
 'Chelsea#H#2-0#A#Atl tico Win Chelsea',
 'Bayern#H#2-1#A#Lazio Win Bayern',
 'Man  City#H#2-0#A#M nchengladbach Win Man  City',
 'Real Madrid#H#3-1#A#Atalanta Win Real Madrid',
 'Liverpool#H#2-0#A#Leipzig Win Liverpool',
 'Paris#H#1-1#A#Barcelona Win Barcelona',
 'Juventus#H#3-2#A#Porto Win Juventus',
 'Dortmund#H#2-2#A#Sevilla Win Sevilla',
 'Atalanta#H#0-1#A#Real Madrid Win Real Madrid',
 'M nchengladbach#H#0-2#A#Man  City Win Man  City',
 'Atl